## This code allows freezing some layers

#### We tried freezing some layers, but the unfrozen model works best, so train with num_frozen_layers=0

In [ ]:
# WandB Authorization Key: 33478b17ae12d300fd1949107f96bf4ab4ff75f9

In [ ]:
!pip install transformers
!pip install wandb --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 13.2 MB/s 
     |████████████████████████████████| 182 kB 58.4 MB/s 
     |████████████████████████████████| 7.6 MB 46.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 15.4 MB/s 
     |████████████████████████████████| 168 kB 60.1 MB/s 
     |████████████████████████████████| 182 kB 69.9 MB/s 
     |████████████████████████████████| 62 kB 1.7 MB/s 
     |████████████████████████████████| 168 kB 62.5 MB/s 
     |████████████████████████████████| 166 kB 62.6 MB/s 
     |████████████████████████████████| 166 kB 74.9 MB/s 
     |████████████████████████████████| 162 kB 70.6 MB/s 
     |████████████████████████████████| 162 kB 41.0 MB/s 
     |████████████████████████████████| 158 kB 74.7 MB/s 
     |████████████████████████████████| 157 kB 59.

### Login to save output to Weights & Biases

In [ ]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
from transformers import DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

import torch
import torch.nn as nn
import torch.nn.functional as F

from torchtext.data.metrics import bleu_score

from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random

from tqdm.auto import tqdm

import wandb

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

### Set Hyperparameters

In [ ]:
config = dict(num_frozen_layers=0,
              learning_rate=5e-5,
              num_epochs=5,
              batch_size=8,
              model='gpt2-medium')  #config['model'] can be any model from the GPT2 family

### Import Model
#### This works for both GPT2 and GPT2 medium

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained(config['model'])
model = GPT2LMHeadModel.from_pretrained(config['model'], pad_token_id=tokenizer.eos_token_id)

tokenizer.pad_token = tokenizer.eos_token

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

In [ ]:
model = model.to(device)

In [ ]:
for name, param in model.named_parameters():
    print(name)

transformer.wte.weight
transformer.wpe.weight
transformer.h.0.ln_1.weight
transformer.h.0.ln_1.bias
transformer.h.0.attn.c_attn.weight
transformer.h.0.attn.c_attn.bias
transformer.h.0.attn.c_proj.weight
transformer.h.0.attn.c_proj.bias
transformer.h.0.ln_2.weight
transformer.h.0.ln_2.bias
transformer.h.0.mlp.c_fc.weight
transformer.h.0.mlp.c_fc.bias
transformer.h.0.mlp.c_proj.weight
transformer.h.0.mlp.c_proj.bias
transformer.h.1.ln_1.weight
transformer.h.1.ln_1.bias
transformer.h.1.attn.c_attn.weight
transformer.h.1.attn.c_attn.bias
transformer.h.1.attn.c_proj.weight
transformer.h.1.attn.c_proj.bias
transformer.h.1.ln_2.weight
transformer.h.1.ln_2.bias
transformer.h.1.mlp.c_fc.weight
transformer.h.1.mlp.c_fc.bias
transformer.h.1.mlp.c_proj.weight
transformer.h.1.mlp.c_proj.bias
transformer.h.2.ln_1.weight
transformer.h.2.ln_1.bias
transformer.h.2.attn.c_attn.weight
transformer.h.2.attn.c_attn.bias
transformer.h.2.attn.c_proj.weight
transformer.h.2.attn.c_proj.bias
transformer.h.2.ln_2

### Freeze Layers

In [ ]:
def create_frozen_model(model, num_frozen_layers):

    layers = ['transformer.h.1','transformer.h.2','transformer.h.3','transformer.h.4','transformer.h.5','transformer.h.6','transformer.h.7','transformer.h.8','transformer.h.9','transformer.h.10','transformer.h.11','transformer.ln_f']

    if num_frozen_layers > len(layers):
        print("Error: Number of layers to freeze should be less than or equal to number of layers in the model:",len(layers))
        return

    keep_layers = layers[num_frozen_layers:]
    for name, param in model.named_parameters():
        param.requires_grad = False   #setting all gradients to false

    for layer_name in keep_layers:
        for name, param in model.named_parameters():
            if name.startswith(layer_name):
                param.requires_grad = True

    return

In [ ]:
create_frozen_model(model, num_frozen_layers=config['num_frozen_layers'])

In [ ]:
for name, param in model.named_parameters():
    if name.startswith('transformer.h.1'):
        print(name)
        print(param.requires_grad)

transformer.h.1.ln_1.weight
True
transformer.h.1.ln_1.bias
True
transformer.h.1.attn.c_attn.weight
True
transformer.h.1.attn.c_attn.bias
True
transformer.h.1.attn.c_proj.weight
True
transformer.h.1.attn.c_proj.bias
True
transformer.h.1.ln_2.weight
True
transformer.h.1.ln_2.bias
True
transformer.h.1.mlp.c_fc.weight
True
transformer.h.1.mlp.c_fc.bias
True
transformer.h.1.mlp.c_proj.weight
True
transformer.h.1.mlp.c_proj.bias
True
transformer.h.10.ln_1.weight
True
transformer.h.10.ln_1.bias
True
transformer.h.10.attn.c_attn.weight
True
transformer.h.10.attn.c_attn.bias
True
transformer.h.10.attn.c_proj.weight
True
transformer.h.10.attn.c_proj.bias
True
transformer.h.10.ln_2.weight
True
transformer.h.10.ln_2.bias
True
transformer.h.10.mlp.c_fc.weight
True
transformer.h.10.mlp.c_fc.bias
True
transformer.h.10.mlp.c_proj.weight
True
transformer.h.10.mlp.c_proj.bias
True
transformer.h.11.ln_1.weight
True
transformer.h.11.ln_1.bias
True
transformer.h.11.attn.c_attn.weight
True
transformer.h.11.

### Create Dataset

In [ ]:
data_file_open = open('/content/data_f.txt', 'r', encoding='UTF-8')
label_file_open = open('/content/label_f.txt', 'r', encoding='UTF-8')
data_lines=data_file_open.readlines()
label_lines = label_file_open.readlines()

input_dataset = []
output_dataset = []
dataset = []


for item in data_lines:
    input_dataset.append(item.strip())
for item in label_lines:
    output_dataset.append(item.strip())

In [ ]:
print(len(input_dataset))
print(len(output_dataset))

42535
42535


In [ ]:
dataset = []
for i in range(len(input_dataset)):
    data = input_dataset[i]+' = '+output_dataset[i]
    dataset.append(data)

In [ ]:
train_dataset, val_dataset = train_test_split(dataset, test_size=0.2)

In [ ]:
print(len(train_dataset))
print(len(val_dataset))

34028
8507


In [ ]:
tokenized_dataset_train = []

tokenized_dataset_val = []


for data in train_dataset:
    tokenized_dataset_train.append(tokenizer.encode(data))

for data in val_dataset:
    tokenized_dataset_val.append(tokenizer.encode(data))

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

### Initialize W&B to save logs

In [ ]:
run_name = config['model']+'_'+str(config['num_frozen_layers'])+'frozen_'+str(config['learning_rate'])+'lr_'+str(config['num_epochs'])+'epochs'

In [ ]:
run_name

'gpt2_0frozen_5e-05lr_5epochs'

In [ ]:
wandb.init(
  project="Antiquator_Trial_new",
  config=config
)

wandb.run.name = run_name

wandb.run.save()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: rohan-ajwani (antiquator). Use `wandb login --relogin` to force relogin


wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


True

In [ ]:
wandb.run.name

'gpt2_10frozen_5e-05lr_5epochs'

### Create Trainer

In [ ]:
training_args = TrainingArguments(
          output_dir='/content/train_2',
          learning_rate=config['learning_rate'],
          per_device_train_batch_size=config['batch_size'],
          per_device_eval_batch_size=config['batch_size'],
          num_train_epochs=config['num_epochs'],
          evaluation_strategy = "steps",
          eval_steps=1000,
          save_strategy='epoch',
          #report_to="wandb"
      )

In [ ]:
trainer = Trainer(
          model=model,
          args=training_args,
          train_dataset=tokenized_dataset_train,
          eval_dataset=tokenized_dataset_val,
          data_collator=data_collator)

### Training the model

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 34028
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 21270
  Number of trainable parameters = 354823168
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss


KeyboardInterrupt: ignored

In [ ]:
trainer.save_model()

Saving model checkpoint to /content/train_2
Configuration saved in /content/train_2/config.json
Model weights saved in /content/train_2/pytorch_model.bin


In [ ]:
model = GPT2LMHeadModel.from_pretrained('/content/train_2')

loading configuration file /content/train_2/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 50256,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "use_cache": true,
  "vocab_siz

### Saving Model to Drive

In [ ]:
from google.colab import drive

import shutil

drive.mount('/content/drive')

In [ ]:
save_name = '/content/'+run_name+'.pt'

In [ ]:
save_name

In [ ]:
torch.save(model.state_dict(), save_name)

In [ ]:
shutil.copy(save_name,"/content/drive/MyDrive/ECE1786_Project/Models")